# 커스텀 데이터셋과 데이터로드를 활용해 CSV에서 훈련 데이터 불러오기

### 문제 목표
커스텀 데이터셋과 데이터로더를 파이토치로 구현해 주어진 `data.csv` 파일에서 데이터를 불러오는 것이 목표이다. 불러온 데이터는 이미 구현된 선형 회귀 모델을 실행하는데 사용될 것이다.

### 요구사항
1. **데이터셋 클래스**:
   - 다음과 같은 `CustomDataset`를 구현한다:
      - `data.csv` 파일에서 데이터를 읽는다.
      - 특징(X)과 대상 값(Y)을 별도로 저장한다.
      - 인덱싱을 위해 PyTorch의 `__len__` 및 `__getitem__` 메서드를 구현한다.

   레퍼런스: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files

2. **데이터로더**:
   - PyTorch의 `DataLoader`를 사용하여 데이터셋을 배치 단위로 반복 가능한 객체로 만든다.
   - 사용자 정의 배치 크기와 데이터 셔플링을 지원한다.
   
   레퍼런스: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#preparing-your-data-for-training-with-dataloaders

In [16]:
import torch
import pandas as pd

torch.manual_seed(42)
X = torch.rand(100, 1) * 10  # 100 data points between 0 and 10
y = 2 * X + 3 + torch.randn(100, 1)  # Linear relationship with noise

# Save the generated data to data.csv
data = torch.cat((X, y), dim=1)
df = pd.DataFrame(data.numpy(), columns=['X', 'y'])
df.to_csv('data.csv', index=False)

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# TODO: Add the missing code
class LinearRegressionDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.x = torch.Tensor(self.data['X'].values).view(-1, 1) # 차원 방향을 바꿔야지 정상적으로 matmul이 가능합니다!
        self.y = torch.Tensor(self.data['y'].values).view(-1, 1) 
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Example usage of the DataLoader
dataset = LinearRegressionDataset('data.csv')
# TODO: Add the missing code
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


리드로써 해야 할 일:

1. torch.tensor와 torch.Tensor의 차이 설명(https://chatgpt.com/c/6796db6f-1a50-8013-afe2-f19cdec38bb3)
    - 일반인이라면 torch.tensor 쓰세요
2. 차원 맞추는거 예제 설명
3. 모델 nn.Module 상속으로 만드는거 설명.

In [21]:
# Define the Linear Regression Model
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)  # Single input and single output

    def forward(self, x):
        return self.linear(x)

# Initialize the model, loss function, and optimizer
model = LinearRegressionModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
epochs = 1000
for epoch in range(epochs):
    for batch_X, batch_y in dataloader:
        # Forward pass
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Log progress every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [100/1000], Loss: 1.5806
Epoch [200/1000], Loss: 0.4760
Epoch [300/1000], Loss: 1.3995
Epoch [400/1000], Loss: 0.4694
Epoch [500/1000], Loss: 0.5121
Epoch [600/1000], Loss: 1.6720
Epoch [700/1000], Loss: 1.2173
Epoch [800/1000], Loss: 0.3319
Epoch [900/1000], Loss: 0.6111
Epoch [1000/1000], Loss: 1.6136


In [13]:
# Display the learned parameters
[w, b] = model.linear.parameters()
print(f"Learned weight: {w.item():.4f}, Learned bias: {b.item():.4f}")

# Testing on new data
X_test = torch.tensor([[4.0], [7.0]])
with torch.no_grad():
    predictions = model(X_test)
    print(f"Predictions for {X_test.tolist()}: {predictions.tolist()}")

Learned weight: 1.9609, Learned bias: 3.2234
Predictions for [[4.0], [7.0]]: [[11.067010879516602], [16.949682235717773]]
